# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [55]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


/home/n1c3/.local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf', 'permutation']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [56]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'HeadBack', 'Knee', 'StandInit', 'Sit', 'Left', 'Crouch', 'Stand', 'Frog', 'Right']


In [57]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    #??? already done?!?
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [58]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in np.arange(0,len(classes)):
    data,target=load_pose_data(i)
    all_data+=(data)
    all_target+=(target)

print(np.array(all_data).shape)    
#Not 200?!? 
#Does all data mean in one list/array? would be more practib
print 'total number of data', len(all_data)

(222, 10)
total number of data 222


In [59]:
# shuffle data
#why not using a more modern way???
#dep. and indepen. data not together?

#@TODO permution on labels

permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
#self
d=np.array(all_data)
T=np.array(all_target)
training_d= d[training_data]
training_t=T[training_data]

n_test=np.setdiff1d(np.arange(0,len(all_data),1),training_data)


test_d=d[n_test]
test_t=T[n_test]


#raining_target=len(n_training_data)

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [60]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [61]:
clf.fit(training_d, training_t) # YOUR CODE HERE

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [62]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([7]), [10])

In [63]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [64]:
expected = []
predicted = []
expected=training_t
predicted=clf.predict(training_d)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00        38
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        14
           7       1.00      1.00      1.00        22
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        10

   micro avg       1.00      1.00      1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [65]:
expected = []
predicted = []

expected=test_t
predicted= clf.predict(test_d)


evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00        14
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         6
           7       0.89      1.00      0.94         8
           8       1.00      1.00      1.00         6
           9       1.00      1.00      1.00         3
          10       0.00      0.00      0.00         1

   micro avg       0.99      0.99      0.99        67
   macro avg       0.90      0.91      0.90        67
weighted avg       0.97      0.99      0.98        67


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

/home/n1c3/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [54]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [53]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([7]), [10])